In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
# #結巴 匯入自己的詞典
# jieba.set_dictionary('./dict/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('./dic/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

In [3]:
#一个商品
review = open('../CategoryCluster/C10/CN/pre/review.csv',encoding='utf-8').read().split('\n')
# review[0] = review[0].replace('\ufeff','')
review[0]
len(review)

551

In [4]:
#一条评论
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
# Reviews[:100]
len(Reviews)

5500

In [5]:
# 断词
import jieba  
import jieba.analyse  
corpus=[]
for review in Reviews:
#     if review !='':
    words=jieba.cut(review, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    corpus.append(pincut)
# corpus =corpus[:-1]   
len(corpus)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/hz/s0m8h80s4bb47624crxjd6h40000gn/T/jieba.cache
Loading model cost 0.960 seconds.
Prefix dict has been built succesfully.


5500

In [6]:
#title
asins = open('../CategoryCluster/C10/CN/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
# ASINs =ASINs[:-1]   
len(ASINs)

5500

In [7]:
#ranks
ranks = open('../CategoryCluster/C10/CN/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split('\t')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

5500

## Tf-idf and document similarity

In [8]:
#将评论中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer(max_df=0.95, max_features=200000,
                                 min_df=0.01)

In [9]:
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

In [10]:
tf = vectorizer.fit_transform(corpus)

In [11]:
#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
tfidf = transformer.fit_transform(tf)

In [12]:
#获取词袋模型中的前10个词语    
word = vectorizer.get_feature_names()
word[:10]

['一只', '一岁', '一点', '不太好', '不好', '不用', '不错', '东西', '乔治', '买贵']

In [13]:
# 词袋模型中共有几个词语
len(word)

127

In [14]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重  
weight = tfidf.toarray() 
weight

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.47507513,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
weight.shape

(5500, 127)

In [16]:
#词频矩阵
tfidframe = pd.DataFrame(np.round(weight, 2))

In [17]:
tfidframe[:10]

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.0,0.0,0.0,0.0,0.0,0.0,0.48,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.57,0.82,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.53,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.59,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## cosine similarity

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf)
dist = 1 - similarity

In [19]:
print(dist)

[[ -2.22044605e-16   1.00000000e+00   1.00000000e+00 ...,   8.21548463e-01
    1.00000000e+00   8.00022691e-01]
 [  1.00000000e+00   0.00000000e+00   6.43828837e-01 ...,   8.51609636e-01
    8.73511322e-01   7.92826753e-01]
 [  1.00000000e+00   6.43828837e-01  -2.22044605e-16 ...,   6.74952986e-01
    1.00000000e+00   4.18332341e-01]
 ..., 
 [  8.21548463e-01   8.51609636e-01   6.74952986e-01 ...,  -2.22044605e-16
    9.70117524e-01   8.10930664e-01]
 [  1.00000000e+00   8.73511322e-01   1.00000000e+00 ...,   9.70117524e-01
   -6.66133815e-16   1.00000000e+00]
 [  8.00022691e-01   7.92826753e-01   4.18332341e-01 ...,   8.10930664e-01
    1.00000000e+00   0.00000000e+00]]


In [20]:
similarity_df = pd.DataFrame(similarity)

In [21]:
similarity_df[:10]

,0,1,2,3,4,5,6,7,8,9,...,5490,5491,5492,5493,5494,5495,5496,5497,5498,5499
0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.275861,0.253932,0.628771,0.000000,...,0.070721,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.178452,0.000000,0.199977
1,0.000000,1.000000,0.356171,0.000000,0.0,0.0,0.000000,0.000000,0.292857,0.000000,...,0.123747,0.0,0.228340,0.0,0.0,0.383206,0.000000,0.148390,0.126489,0.207173
2,0.000000,0.356171,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.280485,0.000000,...,0.000000,0.0,0.218693,0.0,0.0,0.198978,0.000000,0.325047,0.000000,0.581668
3,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.500177,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.239455,0.000000,0.070740,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.275861,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.303214,0.337548,0.000000,...,0.084446,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.238789
7,0.253932,0.000000,0.000000,0.000000,0.0,0.0,0.303214,1.000000,0.310715,0.000000,...,0.077733,0.0,0.000000,0.0,0.0,0.000000,0.231774,0.000000,0.000000,0.219806
8,0.628771,0.292857,0.280485,0.000000,0.0,0.0,0.337548,0.310715,1.000000,0.000000,...,0.086535,0.0,0.179818,0.0,0.0,0.163607,0.000000,0.000000,0.000000,0.407845
9,0.000000,0.000000,0.000000,0.500177,0.0,0.0,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.141430,0.000000


# K-means clustering

In [22]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 538 ms, sys: 30.6 ms, total: 569 ms
Wall time: 461 ms
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [23]:
# from scipy.spatial.distance import cdist
# X = weight #在weight矩阵中元素w[i][j]表示j词在i类文本中的tf-idf权重  
# K = range(1, 100)
# meandistortions = []
# for k in K:
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(X)
#     # 求kmeans的成本函数值
#     meandistortions.append(sum(np.min(cdist(X, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# plt.figure()
# plt.grid(True)
# plt1 = plt.subplot(2,1,1)
# # 画样本点
# plt1.plot(X[:,0],X[:,1],'k.');
# plt2 = plt.subplot(2,1,2)
# # 画成本函数值曲线
# plt2.plot(K, meandistortions, 'bx-')
# plt.show()

In [24]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [25]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters }
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster'])

In [26]:
# frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

In [27]:
# km.inertia_

In [28]:
# grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
# grouped.mean() # 每个聚类的平均排名（0 到 5）

In [29]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
# for i in range(num_clusters):
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() # 空行
#     print() # 空行

In [30]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# # print(order_centroids)
# for i in range(num_clusters):
#     print("Cluster %d:" % i, end='')
#     time.sleep(1)
#     for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
#         print(' %s' % word[ind]  , end=',')
#     print() # 空行
#     print() # 空行

## LDA

In [31]:
from sklearn.decomposition import LatentDirichletAllocation

In [32]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [33]:
%time lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 22.8 s, sys: 43.7 ms, total: 22.9 s
Wall time: 22.9 s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [34]:
%time lda_matrix = lda.fit_transform(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 23.2 s, sys: 62.3 ms, total: 23.3 s
Wall time: 23.3 s


In [35]:
lda_matrix #表示每一個文檔屬於每一個聚類的概率

array([[ 0.025     ,  0.02500134,  0.27499996, ...,  0.02500113,
         0.02500066,  0.52499692],
       [ 0.03333333,  0.69997929,  0.03333789, ...,  0.03334378,
         0.03333644,  0.03333393],
       [ 0.03333333,  0.03334428,  0.36666136, ...,  0.36665684,
         0.03333333,  0.03333751],
       ..., 
       [ 0.00769231,  0.00769274,  0.93076829, ...,  0.00769231,
         0.00769265,  0.00769239],
       [ 0.00588235,  0.0058827 ,  0.00588261, ...,  0.00588235,
         0.00588299,  0.00588235],
       [ 0.01666667,  0.84998302,  0.01667519, ...,  0.0166685 ,
         0.0166672 ,  0.01667233]])

In [36]:
#https://www.pixpo.net/education/0IJdxJRZ.html
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [37]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.025000,0.025001,0.275000,0.025000,0.025000,0.025000,0.025000,0.025001,0.025001,0.524997
1,0.033333,0.699979,0.033338,0.033333,0.033333,0.033335,0.033333,0.033344,0.033336,0.033334
2,0.033333,0.033344,0.366661,0.033333,0.033333,0.033333,0.033333,0.366657,0.033333,0.033338
3,0.366667,0.033333,0.033333,0.366666,0.033333,0.033335,0.033333,0.033333,0.033333,0.033333
4,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [38]:
%time km_lda =km.fit_transform(features)

CPU times: user 131 ms, sys: 13.2 ms, total: 144 ms
Wall time: 137 ms


In [39]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [40]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews})
ldaframe = ldaframe[['title','rank','review']]
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)
ldaClusterframe[:5]

,title,rank,review,ClusterLabel
0,﻿B00X7BH2G4_1,5.0,"""太小了，质量不错，不掉毛""",0
1,B00X7BH2G4_2,5.0,很逼真，也不掉毛，宝宝非常喜欢,2
2,B00X7BH2G4_3,4.0,挺好，孩子挺喜欢的，就是有点小,6
3,B00X7BH2G4_4,3.0,很可爱啊，细节地方很用心。,8
4,B00X7BH2G4_5,3.0,只有一个脏兮兮的塑料袋裹着。回家用数梳子梳一下浮毛，结果属下来好几撮带着胶的毛，顿时觉得心塞...,1


In [41]:
ldaClusterframe['ClusterLabel'].value_counts()

1    2103
2     754
8     596
0     498
7     396
4     343
6     236
3     215
5     195
9     164
Name: ClusterLabel, dtype: int64

In [42]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    4.636546
1    1.219211
2    4.726790
3    3.879070
4    4.673469
5    4.984615
6    4.411017
7    3.851010
8    4.488255
9    4.762195
Name: rank, dtype: float64

In [43]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,review,ClusterLabel
0,﻿B00X7BH2G4_1,5.0,"""太小了，质量不错，不掉毛""",0
4429,B00RUEQX14_10,5.0,还不错，搂着不错，抱着睡觉,0
4422,B00RUEQX14_3,4.0,不错，女朋友觉得有点丑，哈哈，其实我也就觉得毛发那里是丑了点,0
4420,B00RUEQX14_1,3.0,"""远看不错，近看各种抽线，质量对不起它的名气呀。""",0
4418,B01BD7T79I_9,4.0,一般，大号的开裂了一点,0
4411,B01BD7T79I_2,4.0,我买L号猪是在专卖店买的，要300多，这里的价格就划算多了，手感都一样，就是味道比较大，丢在...,0
1132,B01EARLUBG_3,2.0,官方的app连不上，需要科学上网，质量也没有前一代那么好，我的已经有点坏掉了。,0
4389,B00RUER1PG_10,5.0,还不错，搂着不错，抱着睡觉,0
4382,B00RUER1PG_3,4.0,不错，女朋友觉得有点丑，哈哈，其实我也就觉得毛发那里是丑了点,0
4380,B00RUER1PG_1,3.0,"""远看不错，近看各种抽线，质量对不起它的名气呀。""",0


In [44]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
#     print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        
        
    print() # 空行
    print() # 空行

 颜色, 蓝色, 图片, 退货, 尺寸, 购买, 推荐, 细节, 袋子, 标签, 外面, 长些, 强太多, 清晰, 官旗, 长度, 一点, 亚马逊, 超级, 手感, 买贵, 感觉, 收到, 玩偶, 好看, 一只, 质量, 介绍, 只能, 玩具,

 喜欢, 手感, 宝宝, 做工, 不错, 朋友, 玩偶, 玩具, 天天, 孩子, 第一个, 耐脏, 白色, 一岁, 半岁, 一只, 女儿, 特别, 精致, 感觉, 柔和, 送给, 生日礼物, 尺寸, 购买, 舒服, 超级, 毛绒玩具, 材质, 大小,

 孩子, 太小, 一岁, 适合, 宝宝, 小朋友, 还会, 危险, 看不懂, 窒息, 摸摸, 感觉, 女儿, 玩具, 喜欢, 我家, 袋子, 做工, 介绍, 可爱, 一点, 味道, 蓝色, 睡觉, 美国, 活动, 只能, 收到, 颜色, 细节,

 可爱, 价格, 特别, 真的, 女儿, 开心, 很软, 生日礼物, 亚马逊, 只能, 不好, 玩具, 电池, 第一个, 舒服, 手感, 送给, 喜欢, 一只, 收到, 超级, 尺寸, 我家, 宝宝, 感觉, 做工, 朋友, 蓝色, 柔软, 大小,

 积分, 不用, 评价, 评论, 推荐, 发出, 好好, 复制, 走到, 换钱, 浪费, 省事, 既能, 这段话, 复制到, 只能, 感觉, 品牌, 活动, 小孩, 睡觉, 亚马逊, 材质, 简单, 手感, 小朋友, 价格, 质量, 美亚, 玩具,

 超级, 毛绒, 送给, 手感, 礼物, 可爱, 包装, 简单, 小兔子, 儿童节, 宝宝, 好好, 介绍, 柔软, 白色, 撕扯, 分包, 包装纸, 盒内, 时式, 广告宣传, 放入, 兔纸, 小尾巴, 最棒, 软绵绵, 一只, 推荐, 玩具, 生日礼物,

 东西, 舒服, 包装, 电池, 不好, 送人, 瑕疵, 感觉, 不太好, 小孩, 只能, 拿到, 介绍, 一点, 柔和, 亚马逊, 玩具, 美国, 味道, 宝宝, 喜欢, 价格, 一只, 超级, 孩子, 退货, 手感, 质量, 不错, 送给,

 喜欢, 美亚, 大小, 粉色, 好看, 爱不释手, 小家伙, 质量, 最早, 紫色, 乔治, 女儿, 评论, 精致, 收到, 真的, 拿到, 礼物, 品牌, 开心, 小孩, 可爱, 孩子, 小朋友, 儿童节, 玩具, 毛绒, 美国, 

In [45]:
order_centroids

array([[126, 107,  29, ...,  58,  47,  90],
       [ 28,  56,  41, ...,  64,  90,  31],
       [ 39,  35,   1, ...,  58,  64, 111],
       ..., 
       [ 28, 103,  33, ...,  95,  31, 119],
       [ 71,  62,  93, ...,  78,  95, 114],
       [  6, 112,  10, ...,  64,  78,  30]])

In [46]:
ftf = transformer.fit_transform(order_centroids)

In [47]:
ftf = ftf.toarray()

In [48]:
#词频矩阵
tfidMMframe = pd.DataFrame(np.round(ftf, 2))
tfidMMframe[:5]

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.15,0.13,0.04,0.14,0.06,0.14,0.07,0.12,0.13,0.09,...,0.08,0.04,0.11,0.09,0.04,0.03,0.14,0.07,0.06,0.11
1,0.03,0.07,0.05,0.02,0.01,0.08,0.10,0.10,0.04,0.05,...,0.04,0.09,0.11,0.03,0.07,0.06,0.09,0.08,0.11,0.04
2,0.05,0.04,0.00,0.15,0.05,0.06,0.14,0.03,0.11,0.12,...,0.09,0.11,0.09,0.04,0.14,0.12,0.14,0.07,0.08,0.13
3,0.03,0.01,0.10,0.11,0.04,0.06,0.06,0.10,0.01,0.03,...,0.04,0.03,0.03,0.11,0.07,0.09,0.14,0.11,0.14,0.04
4,0.11,0.01,0.13,0.13,0.07,0.03,0.04,0.04,0.14,0.07,...,0.03,0.10,0.06,0.09,0.03,0.11,0.15,0.12,0.07,0.13


In [49]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        print(' %f' % lda.components_[i][ind-1], end=',')
        print(' %f' % (tfidMMframe[i:i+1][ind]), end=',')#a词在所属群中出现的频率
        print(' %f' % max(tfidframe[:][ind]), end=',') #a词与所有顾客评论中出现最高词频的相对指数
        print(' %f' % (tfidMMframe[i:i+1][ind]/max(tfidframe[:][ind])), end='\n')
        
    print('') # 空行
    print() # 空行

Cluster 0:
 颜色, 67.534786, 0.110000, 1.000000, 0.110000
 蓝色, 0.100003, 0.110000, 0.910000, 0.120879
 图片, 0.100004, 0.100000, 1.000000, 0.100000
 退货, 0.100000, 0.090000, 0.920000, 0.097826
 尺寸, 0.100000, 0.090000, 1.000000, 0.090000
 购买, 0.100004, 0.090000, 0.850000, 0.105882
 推荐, 0.100000, 0.010000, 1.000000, 0.010000
 细节, 0.100000, 0.080000, 0.870000, 0.091954
 袋子, 136.009083, 0.110000, 0.680000, 0.161765
 标签, 0.100002, 0.060000, 1.000000, 0.060000
 外面, 0.100000, 0.080000, 0.740000, 0.108108
 长些, 0.100001, 0.070000, 0.290000, 0.241379
 强太多, 0.100000, 0.110000, 0.290000, 0.379310
 清晰, 0.100000, 0.020000, 0.290000, 0.068966
 官旗, 0.100003, 0.050000, 0.290000, 0.172414
 长度, 67.534786, 0.060000, 0.290000, 0.206897
 一点, 0.100000, 0.040000, 1.000000, 0.040000
 亚马逊, 0.100006, 0.040000, 1.000000, 0.040000
 超级, 0.100000, 0.030000, 0.820000, 0.036585
 手感, 0.100001, 0.150000, 1.000000, 0.150000
 买贵, 0.100000, 0.090000, 0.560000, 0.160714
 感觉, 0.100000, 0.010000, 1.000000, 0.010000
 收到, 0.100000, 

 喜欢, 0.100014, 0.010000, 1.000000, 0.010000
 美亚, 0.100003, 0.060000, 0.770000, 0.077922
 大小, 0.100001, 0.150000, 1.000000, 0.150000
 粉色, 0.100002, 0.080000, 0.860000, 0.093023
 好看, 0.100000, 0.120000, 1.000000, 0.120000
 爱不释手, 0.100000, 0.120000, 0.840000, 0.142857
 小家伙, 0.100012, 0.110000, 0.900000, 0.122222
 质量, 0.100000, 0.040000, 1.000000, 0.040000
 最早, 0.100000, 0.100000, 0.390000, 0.256410
 紫色, 39.367522, 0.070000, 0.390000, 0.179487
 乔治, 0.100001, 0.080000, 1.000000, 0.080000
 女儿, 0.100001, 0.090000, 1.000000, 0.090000
 评论, 0.100000, 0.110000, 0.810000, 0.135802
 精致, 98.472386, 0.110000, 1.000000, 0.110000
 收到, 0.100000, 0.060000, 1.000000, 0.060000
 真的, 0.100000, 0.020000, 1.000000, 0.020000
 拿到, 0.100003, 0.100000, 1.000000, 0.100000
 礼物, 0.100003, 0.140000, 1.000000, 0.140000
 品牌, 0.100003, 0.140000, 0.700000, 0.200000
 开心, 0.100000, 0.040000, 1.000000, 0.040000
 小孩, 0.100000, 0.110000, 1.000000, 0.110000
 可爱, 0.100001, 0.110000, 1.000000, 0.110000
 孩子, 90.322328, 0.060000, 1

In [50]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      15.556914        1       1 -0.280593 -0.043213
5      12.642638        1       2  0.121051  0.054191
9      12.136797        1       3 -0.190201 -0.006039
8      10.452329        1       4 -0.094381  0.255019
7       9.283330        1       5 -0.200699 -0.274109
4       9.098470        1       6  0.229951  0.021074
0       8.489737        1       7  0.182601  0.069008
3       7.706957        1       8  0.114703 -0.252618
2       7.510003        1       9 -0.096882  0.247773
6       7.122825        1      10  0.214449 -0.071086, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
28    Default  1333.000000    喜欢  1333.000000  30.0000  30.0000
25    Default   725.000000    可爱   725.000000  29.0000  29.0000
6     Default   914.000000    不错   914.000000  28.0000  28.0000
39    Default   495.000000    孩子   495.000000  27.0000  27.0000
112   Default   453.000000    质量   453.000000  26.0000  26.0000
7     Default   258.000000    东西   258.000000  25.0000  25.0000
95    Default   250.000000    积分   250.000000  24.0000  24.0000
71    Default   312.000000    柔软   312.000000  23.0000  23.0000
56    Default   408.000000    手感   408.000000  22.0000  22.0000
10    Default   300.000000   亚马逊   300.000000  21.0000  21.0000
41    Default   580.000000    宝宝   580.000000  20.0000  20.0000
13    Default   269.000000    做工   269.000000  19.0000  19.0000
5     Default   189.000000    不用   189.000000  18.0000  18.0000
1     Default   287.000000    一岁   287.000000  17.0000  17.0000
116   Default   215.000000    超级   215.000000  16.0000  16.0000
106   Default   167.000000    舒服   167.000000  15.0000  15.0000
35    Default   163.000000    太小   163.000000  14.0000  14.0000
109   Default   170.000000    评价   170.000000  13.0000  13.0000
126   Default   211.000000    颜色   211.000000  12.0000  12.0000
18    Default   202.000000    包装   202.000000  11.0000  11.0000
81    Default   185.000000    特别   185.000000  10.0000  10.0000
12    Default   147.000000    价格   147.000000   9.0000   9.0000
123   Default   177.000000    适合   177.000000   8.0000   8.0000
107   Default   142.000000    蓝色   142.000000   7.0000   7.0000
74    Default   167.000000    毛绒   167.000000   6.0000   6.0000
92    Default   158.000000    真的   158.000000   5.0000   5.0000
59    Default   254.000000    推荐   254.000000   4.0000   4.0000
68    Default   178.000000    朋友   178.000000   3.0000   3.0000
2     Default   178.000000    一点   178.000000   2.0000   2.0000
29    Default   124.000000    图片   124.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
11    Topic10    13.993481    介绍    89.312489   0.7883  -4.5194
2     Topic10    13.549940    一点   178.389105   0.0643  -4.5517
85    Topic10     0.100757  生日礼物    53.216760  -3.6275  -9.4531
26    Topic10     0.100772    味道    57.420149  -3.7034  -9.4529
75    Topic10     0.100762  毛绒玩具    58.194688  -3.7169  -9.4530
8     Topic10     0.100759    乔治    59.038504  -3.7313  -9.4531
0     Topic10     0.100769    一只    59.165947  -3.7334  -9.4530
104   Topic10     0.100772    美国    61.273949  -3.7684  -9.4529
70    Topic10     0.100812    柔和    61.360941  -3.7694  -9.4525
52    Topic10     0.100761    很软    61.442430  -3.7712  -9.4530
20    Topic10     0.100757    半岁    62.483234  -3.7881  -9.4531
27    Topic10     0.100760    品牌    63.737308  -3.8079  -9.4530
105   Topic10     0.100757    耐脏    64.230958  -3.8157  -9.4531
101   Topic10     0.100757    紫色    66.162020  -3.8453  -9.4531
66    Topic10     0.100757    最早    66.162020  -3.8453  -9.4531
50    Topic10     0.100762    开心    66.907182  -3.8564  -9.4530
97    Topic10     0.100757   第一个    68.192741  -3.8755  -9.4531
53    Topic10     0.100761    想象    69.259041  -3.8910  -9.4530
10    Topic10     0.100787   亚马逊   300.158009

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 02:19:35.823837. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [51]:
# #挑選特定條件(抽樣)
# samples = ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 4][:441]
# Rsamples = samples['review'].values
# Rsamples

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 02:19:37.071186. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [52]:
# with open('../test.csv','w',encoding='utf-8') as f :
#     f.write(str(Rsamples))

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-03-03 02:19:37.079486. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
